In [7]:
import pandas as pd

# Load the data from the uploaded Excel file
data_path = 'C:\MyProject\MIMIC-IPE\加了中介变量的数据集\贝叶斯数据集.xlsx'
data = pd.read_excel(data_path)

# Display the first few rows of the dataframe and a summary of the data
data.head(), data.info(), data.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2039 entries, 0 to 2038
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Subject_id             2039 non-null   int64  
 1   Hadm_id                2039 non-null   int64  
 2   Group_Label            2039 non-null   object 
 3   Gender                 588 non-null    object 
 4   Weight_admit           572 non-null    float64
 5   Height                 239 non-null    float64
 6   First_hospitalization  2039 non-null   int64  
 7   Time_of_death_months   2039 non-null   float64
 8   Death_event            2039 non-null   int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 143.5+ KB


(   Subject_id   Hadm_id Group_Label Gender  Weight_admit  Height  \
 0    17618653  25626055      DVT-PE    NaN           NaN     NaN   
 1    12486000  21105236         IPE    NaN           NaN     NaN   
 2    12208701  20368865      DVT-PE    NaN           NaN     NaN   
 3    17650347  28346420      DVT-PE    NaN           NaN     NaN   
 4    15566220  22593658         IPE    NaN           NaN     NaN   
 
    First_hospitalization  Time_of_death_months  Death_event  
 0                      0              0.131406            1  
 1                      0              5.026281            0  
 2                      1              5.026281            0  
 3                      1              5.026281            0  
 4                      1              5.026281            0  ,
 None,
          Subject_id       Hadm_id  Weight_admit      Height  \
 count  2.039000e+03  2.039000e+03    572.000000  239.000000   
 mean   1.501044e+07  2.495140e+07     91.469580  170.907950   
 std  

In [11]:
import pymc3 as pm
import numpy as np

# Data preprocessing
# Handle categorical data for Gender (binary encoding: Male=0, Female=1)
data['Gender_encoded'] = data['Gender'].map({'M': 0, 'F': 1})

# Dropping the original Gender column for simplicity in modeling
data.drop('Gender', axis=1, inplace=True)

# Fill NA values for modeling (temporary fill for initial setup)
data['Weight_admit'].fillna(data['Weight_admit'].mean(), inplace=True)
data['Height'].fillna(data['Height'].mean(), inplace=True)

# Setting up the Bayesian Model for Gender imputation
with pm.Model() as model:
    # Priors for the mean and standard deviation of weights and heights
    weight_mean = pm.Normal('weight_mean', mu=data['Weight_admit'].mean(), sigma=20)
    height_mean = pm.Normal('height_mean', mu=data['Height'].mean(), sigma=10)

    # Priors for weight and height given gender
    weight_sd = pm.HalfNormal('weight_sd', sigma=10)
    height_sd = pm.HalfNormal('height_sd', sigma=5)

    # Gender model: P(Gender | Weight, Height)
    gender_p = pm.math.invlogit(weight_mean + height_mean)
    gender = pm.Bernoulli('gender', p=gender_p, observed=data['Gender_encoded'])

    # Likelihoods for weight and height
    weight = pm.Normal('weight', mu=weight_mean, sigma=weight_sd, observed=data['Weight_admit'])
    height = pm.Normal('height', mu=height_mean, sigma=height_sd, observed=data['Height'])

    # Prior for Gender based on weights and heights (model to impute missing genders)
    trace = pm.sample(500, return_inferencedata=False)

# Checking the output from the model
trace_summary = pm.summary(trace)
trace_summary


ImportError: cannot import name 'local_bitwidth' from 'theano.configdefaults' (C:\Users\ZhouNan\anaconda3\envs\NN\Lib\site-packages\theano\configdefaults.py)